# Christine's Code: CNN Introduction

In [12]:
import tensorflow as tf
# from keras.preprocessing.image import ImageDataGenerator

In [7]:
# Keras now lives under TF
a = tf.keras.preprocessing.image.ImageDataGenerator()

In [8]:
# 4,000 images of dogs and 4,000 images of cats
# Total of 8,000 images

# Juypter notebook - run code via your machine with VSCode
# Google Colab will fail for highly intensive code that requires large datasets


In [9]:
# Goal is to deploy so it can differentiate between 2 different images
# Dog and Cat

In [10]:
tf.__version__

'2.17.0'

# Data Preprocessing

In [ ]:
# We need to apply transformations to avoid overfitting
# high accuracy on training set, low accuracy in test set
# Why we do this - avoid overfitting

# Image Augmentation
# Your CNN does not overlearn

# Example - horizontal flips, zooms, rotations

In [16]:
# Create Preprocessor for Training Set
# Create instance, and create training class generator
training_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255, # feature scaling
    shear_range = 0.2, # 
    zoom_range = 0.2, # Zoom into the picture
    horizontal_flip = True # flip horizontally
    )


# Create the training set
# Flow from directory - tell it where the dataset for training is coming from
training_set = training_datagen.flow_from_directory('/Users/christine/VSCode/deeplearningcourse_remote/deeplearningcourse/Part 2 - CNN/training_set',
                                                 target_size = (64, 64), # affects resizing
                                                 batch_size = 32,
                                                 class_mode = 'binary') # this is a binary outcome, cat or dog

# Notice the folders are labelled dog and cat

Found 8000 images belonging to 2 classes.


In [18]:
# Create instance, and create test class generator
# DO NOT APPLY THE TRANSFORMATIONS FOR TEST-GEN EXCEPT FOR SCALING

# Test image - you do not want to apply transformations, to prevent over-learning
# i.e. not applying fit_transform() to test set, you only transform()

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/Users/christine/VSCode/deeplearningcourse_remote/deeplearningcourse/Part 2 - CNN/test_set',
                                            target_size = (64, 64), # must be the same size and parameters here
                                            batch_size = 32,
                                            class_mode = 'binary')

# 2000 images for test set
# 8000 images for training set

Found 2000 images belonging to 2 classes.


# Part 1: Convolution and RELU

# Part 2: Max Pooling

# Part 3: Flatten

# Part 4: Full Connection